## Libraries

This is a notebook for compressing your images via compressAI model and quality

In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         #print(os.path.join(dirname, filename))
#         pass

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torch-summary
!pip install torch-lr-finder

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from PIL import Image
from IPython.display import display
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary
from tqdm import tqdm
import torchvision.models as models
import PIL
from torchvision.models import resnet50, ResNet50_Weights
import timm

In [ ]:
torch.manual_seed(0)

## Import of Cheng

In [ ]:
%pip install "numpy<2" matplotlib torch==2.2.2 torchaudio==2.2.2 torchvision==0.17.2 torchmetrics scikit-image kornia

In [ ]:
!pip install compressai

In [ ]:
import torch
from kornia.losses import PSNRLoss, SSIMLoss
from torchmetrics.image import VisualInformationFidelity
import torch.nn.functional as F
from compressai.zoo import models
from torchvision import transforms
import PIL
import numpy as np
from skimage import io , img_as_float

In [ ]:
from compressai.zoo import models
model_names = []
for model_name in models:
    model_names.append(model_name)
print(model_names)

['bmshj2018-factorized', 'bmshj2018-factorized-relu', 'bmshj2018-hyperprior', 'mbt2018-mean', 'mbt2018', 'cheng2020-anchor', 'cheng2020-attn', 'bmshj2018-hyperprior-vbr', 'mbt2018-mean-vbr', 'mbt2018-vbr', 'hrtzxf2022-pcc-rec', 'sfu2023-pcc-rec-pointnet', 'sfu2024-pcc-rec-pointnet2-ssg', 'ssf2020']


In [ ]:
max_values_of_quality = [0] * len(models)

for i, model_name in enumerate(models):
    model_class = models[model_name]
    print(f"Testing {model_name}")

    j = 1
    max_quality = None

    while max_quality is None:
        try:
            print(f"Trying quality={j}")
            model = model_class(quality=j, pretrained=True)
            j += 1
        except ValueError as e:
            # We've found the limit when ValueError is raised
            max_quality = j - 1
            print(f"Max quality for {model_name}: {max_quality}")
        except TypeError as e:
            # This model might not use quality parameter
            print(f"Model {model_name} may not use quality parameter: {e}")
            max_quality = "N/A"
        except Exception as e:
            # Handle other unexpected errors
            print(f"Error with {model_name}: {e}")
            max_quality = "Error"
        if j>20:
            break

    max_values_of_quality[i] = max_quality

Testing bmshj2018-factorized
Trying quality=1


Downloading: "https://compressai.s3.amazonaws.com/models/v1/bmshj2018-factorized-prior-1-446d5c7f.pth.tar" to /root/.cache/torch/hub/checkpoints/bmshj2018-factorized-prior-1-446d5c7f.pth.tar
100%|██████████| 11.5M/11.5M [00:01<00:00, 7.84MB/s]
Downloading: "https://compressai.s3.amazonaws.com/models/v1/bmshj2018-factorized-prior-2-87279a02.pth.tar" to /root/.cache/torch/hub/checkpoints/bmshj2018-factorized-prior-2-87279a02.pth.tar


Trying quality=2


100%|██████████| 11.5M/11.5M [00:01<00:00, 8.64MB/s]
Downloading: "https://compressai.s3.amazonaws.com/models/v1/bmshj2018-factorized-prior-3-5c6f152b.pth.tar" to /root/.cache/torch/hub/checkpoints/bmshj2018-factorized-prior-3-5c6f152b.pth.tar


Trying quality=3


...

Trying quality=8


Downloading: "https://compressai.s3.amazonaws.com/models/v1/ssf2020-mse-8-26439e20.pth.tar" to /root/.cache/torch/hub/checkpoints/ssf2020-mse-8-26439e20.pth.tar
100%|██████████| 134M/134M [00:09<00:00, 14.6MB/s]


Trying quality=9


Downloading: "https://compressai.s3.amazonaws.com/models/v1/ssf2020-mse-9-e89345c4.pth.tar" to /root/.cache/torch/hub/checkpoints/ssf2020-mse-9-e89345c4.pth.tar
100%|██████████| 134M/134M [00:11<00:00, 12.7MB/s]


Trying quality=10
Max quality for ssf2020: 9


In [ ]:
print(max_values_of_quality)
model_names = [x for x in models]
accepted_model =[]
for i in range(14):
    # print(max_values_of_quality[i],model_names[i])
    if type(max_values_of_quality[i]) == int:
        accepted_model.append([model_names[i],max_values_of_quality[i]])
print(accepted_model)

[8, 8, 8, 8, 8, 6, 6, None, None, 'Error', 'N/A', 'N/A', 'N/A', 9]
[['bmshj2018-factorized', 8], ['bmshj2018-factorized-relu', 8], ['bmshj2018-hyperprior', 8], ['mbt2018-mean', 8], ['mbt2018', 8], ['cheng2020-anchor', 6], ['cheng2020-attn', 6], ['ssf2020', 9]]


In [ ]:
# !unzip "/content/val copy.zip"

In [ ]:
!unzip "/content/usimages.zip"

Archive:  /content/usimages.zip
   creating: usimages/
  inflating: usimages/pa958.png      
  inflating: usimages/ms527.png      
  inflating: usimages/wy716.png      
  inflating: usimages/or1623.png     
  inflating: usimages/ky729.png      
  inflating: usimages/de1057.png     
  inflating: usimages/nj1238.png     
  inflating: usimages/or270.png      
  inflating: usimages/nv952.png      
  inflating: usimages/wa1439.png     
  inflating: usimages/nh846.png      
  inflating: usimages/nj937.png      
  inflating: usimages/fl1167.png     
  inflating: usimages/ar810.png      
  inflating: usimages/wi1588.png     
  inflating: usimages/de681.png      
  inflating: usimages/mi1155.png     
  inflating: usimages/ri583.png      
  inflating: usimages/ut1091.png     
  inflating: usimages/co918.png      
  inflating: usimages/nh932.png      
  inflating: usimages/ia215.png      
  inflating: usimages/va82.png       
  inflating: usimages/ny1255.png     
  inflating: usimages/fl830.png  

In [ ]:
vif = VisualInformationFidelity()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # setting the computation on GPU or CPU
vif.to(device)

# x and y are torch images, maximum_value is 1 or 255
# bring the image into GPU or CPU
# x = x.to(device)
# y = y.to(device)

def PSNR(x,y,max_val):
    psnr = PSNRLoss(max_val=max_val)
    return -psnr(x,y).item()
def SSIM(x,y):
    ssim_loss = SSIMLoss(window_size=11, reduction='mean')
    ssim_value = 1-ssim_loss(x,y).item()
    return ssim_value

def BPP(image_tensor,model, num_pixels):
    output = model(image_tensor)
    # Calculate BPP
    bpp = sum(
        torch.log(likelihoods).sum() / (-torch.log(2) * num_pixels)
        for likelihoods in output["likelihoods"].values()
    )
    return bpp.item()

def VIF(x,y):
    return vif(x,y).item()

def mse(x,y):
    return F.mse_loss(x,y).item()
def mae(x,y):
    return F.l1_loss(x,y).item()
def calculate_metrics(x,y,max_val,model,num_pixels):
    num_pixels = x.shape[2]*x.shape[3]
    output = model(x)
    bpp = sum(
        torch.log(likelihoods).sum() / (-torch.log(torch.tensor(2)) * num_pixels)
        for likelihoods in output["likelihoods"].values()
    )
    bpp = bpp.item()
    return {"psnr":PSNR(x,y,max_val),
            "ssim":SSIM(x,y),
            "vif":VIF(x,y),
            "mse":mse(x,y),
            "mae":mae(x,y),
            "bpp":bpp}
def compression_image_tensor(x,model,device,quality):
    x = x.to(device)
    with torch.no_grad():
        compressed = model(x)
        decompressed = compressed["x_hat"]
    return decompressed.detach()


## Compression Pipeline

Cheng2020 Requirement: Input dimensions (height × width) must be multiples of 128.

Your Dataset: 128×224 (height=128 ✔️, width=224 ❌ → 224 ÷ 128 = 1.75)

Kodak Dataset: Likely already uses dimensions like 512×768 (both divisible by 128) → no padding needed.

In [ ]:
import torch
from compressai.zoo import models
from torchvision import transforms
import torchvision.transforms.functional as F
import PIL
import numpy as np
from skimage import io , img_as_float
import pathlib

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # setting the computation on GPU or CPU
print("Device: ", device)

Device:  cuda


In [ ]:
# Original: './image_compression/ISIC-skin-cancer/Test'
data_path = '/content/val'  # From your existing code
set_type = 'test'  # or 'train' depending on your need

# for state in class_name:  # class_name contains your state names
#     plates_path = []
#     image_dir = os.path.join(data_path, set_type, state)



In [ ]:
import os
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import torch.nn.functional as F
from compressai.zoo import models

# Constants
DATA_PATH = "/content/usimages"  # Folder directly containing the images

COMPRESSED_ROOT = os.path.join(DATA_PATH, "compressed")  # Output folder

# Padding function
def pad_to_multiple(img, mul=64):
    w, h = img.size
    pad_width = (mul - w % mul) % mul
    pad_height = (mul - h % mul) % mul
    img_tensor = transforms.ToTensor()(img)
    padded_img = F.pad(img_tensor, (0, pad_width, 0, pad_height), value=0)
    return transforms.ToPILImage()(padded_img)

# Preprocess image
def preprocess_image(image_path):
    img = Image.open(image_path).convert('RGB')
    img = pad_to_multiple(img)
    return transforms.ToTensor()(img).unsqueeze(0)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Clean previous outputs
if os.path.exists(COMPRESSED_ROOT):
    for root, dirs, files in os.walk(COMPRESSED_ROOT, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))

# Gather all images from val/
image_paths = [
    os.path.join(DATA_PATH, f)
    for f in os.listdir(DATA_PATH)
    if f.lower().endswith((".jpg", ".png"))
]

# Preprocess all images
image_tensors = [preprocess_image(p).to(device) for p in image_paths]

# Compression and decompression
for model_name, quality in [("cheng2020-anchor", 1), ("cheng2020-attn", 1),("cheng2020-anchor", 3), ("cheng2020-attn", 3),("cheng2020-anchor", 6), ("cheng2020-attn", 6)]:
    model = models[model_name](quality=quality, pretrained=True).to(device)
    model.eval()

    # Create output subdirectory
    model_output_dir = os.path.join(COMPRESSED_ROOT, f"{model_name}-q{quality}")
    os.makedirs(model_output_dir, exist_ok=True)

    for idx, tensor in enumerate(image_tensors):
        with torch.no_grad():
            compressed = model(tensor)
            decompressed = compressed["x_hat"].clamp(0, 1)

            # Convert and save
            decompressed_np = (decompressed.squeeze().cpu().permute(1, 2, 0).numpy() * 255).astype(np.uint8)
            output_filename = os.path.splitext(os.path.basename(image_paths[idx]))[0] + ".jpg"
            Image.fromarray(decompressed_np).save(os.path.join(model_output_dir, output_filename))

    del model
    torch.cuda.empty_cache()

In [ ]:
#run this before zip and download if use colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!zip -r /content/compressed_full.zip /content/usimages/compressed

  adding: content/usimages/compressed/ (stored 0%)
  adding: content/usimages/compressed/cheng2020-attn-q1/ (stored 0%)
  adding: content/usimages/compressed/cheng2020-attn-q1/ma246.jpg (deflated 3%)
  adding: content/usimages/compressed/cheng2020-attn-q1/wi580.jpg (deflated 4%)
  adding: content/usimages/compressed/cheng2020-attn-q1/oh1158.jpg (deflated 3%)
  adding: content/usimages/compressed/cheng2020-attn-q1/ms1447.jpg (deflated 4%)
  adding: content/usimages/compressed/cheng2020-attn-q1/ar477.jpg (deflated 5%)
  adding: content/usimages/compressed/cheng2020-attn-q1/nv831.jpg (deflated 4%)
  adding: content/usimages/compressed/cheng2020-attn-q1/me775.jpg (deflated 2%)
  adding: content/usimages/compressed/cheng2020-attn-q1/ms478.jpg (deflated 3%)
  adding: content/usimages/compressed/cheng2020-attn-q1/nv497.jpg (deflated 6%)
  adding: content/usimages/compressed/cheng2020-attn-q1/az758.jpg (deflated 3%)
  adding: content/usimages/compressed/cheng2020-attn-q1/tn1659.jpg (deflated 3